In [1]:
import os
from os.path import join
import csv
import cv2
import numpy as np

from keras.layers import Input, Conv2D, Dropout, MaxPool2D, GlobalAveragePooling2D, GlobalMaxPooling2D, LeakyReLU
from keras.layers import concatenate, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from paths import PATH_DATA, PATH_PROJECT
from iterators import roundrobin, repeat_infinitely

Using TensorFlow backend.


In [2]:
class DarknetBlock:
    def __init__(self, filters1, filters3, strides=(1, 1)):
        self._filters1 = filters1
        self._filters3 = filters3
        self._strides = strides
        
    def __call__(self, input_layer, *args, **kwargs):
        x = input_layer
        for filters, kernel, strides in zip([self._filters1, self._filters3],
                                            [(1, 1), (3, 3)],
                                            [(1, 1), self._strides]):
            x = Conv2D(filters, kernel, padding='same', strides=strides)(x)
            x = LeakyReLU()(x)
        return x


def build_model(class_count, is_train=False):
    input_tensor = Input(shape=(1, None, None))
    x = Conv2D(10, (3, 3), strides=(2, 2), padding='same')(input_tensor)
    filters = [20, 30, 40]
    for idx, filter_size in enumerate(filters):
        x = DarknetBlock(filter_size, filter_size * 2)(x)
        x = DarknetBlock(filter_size, filter_size * 2)(x)
        if idx < len(filters) - 1:
            x = MaxPool2D(pool_size=(2, 2), padding='same')(x)
            if is_train:
                x = Dropout(0.1)(x)
    
    x = Conv2D(filters[-1], (1, 1), padding='same')(x)
    max_output = GlobalMaxPooling2D()(x)
    average_output = GlobalAveragePooling2D()(x)
    x = concatenate(inputs=[max_output, average_output], axis=1)
    x = Dense(class_count, activation='softmax')(x)
    return Model(inputs=[input_tensor], outputs=[x])

In [3]:
CLASS_COUNT = 6
# cannot make bigger batches because images are of different size
BATCH_SIZE = 1
TRAIN_SIZE = 7000 * 80 // 100
VAL_SIZE = 7000 - TRAIN_SIZE

In [4]:
def read_label_map(path):
    with open(path, 'r') as fin:
        reader = csv.reader(fin, delimiter=',')
        # ignore header
        next(reader)
        label_map = {filename: int(label) for filename, label in reader}
    return label_map


def get_images_paths(root, label_count):
    dir_paths = list(map(lambda label: join(root, str(label)), range(label_count)))
    file_paths = [list(map(lambda filename: join(dir_path, filename), os.listdir(dir_path))) for dir_path in dir_paths]
    return roundrobin(*file_paths)


def get_train_data(path_images, label_count, label_map):
    paths = repeat_infinitely(get_images_paths, path_images, label_count)
    for path in paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f'Image cannot be read: {path}')
        yield (np.expand_dims(image, 0),
               to_categorical(label_map[os.path.basename(path)], label_count)[0])


def collect_batches(iterable, batch_size=32, randomize=False, probability=0.5, rotate=False):
    while True:
        images, labels = [], []
        while len(images) < batch_size:
            image, label = next(iterable)
            if not randomize or np.random.rand() < probability:
                if rotate:
                    image = np.rot90(image, np.random.randint(0, 4), (1, 2))
                images.append(image)
                labels.append(label)
        yield np.array(images), np.array(labels)

In [5]:
label_map = read_label_map(join(PATH_DATA, 'train_labels.csv'))
gen_train_data = get_train_data(join(PATH_DATA, 'train'), CLASS_COUNT, label_map)
gen_train_batches = collect_batches(gen_train_data, BATCH_SIZE, randomize=True, rotate=True)

gen_val_data = get_train_data(join(PATH_DATA, 'validation'), CLASS_COUNT, label_map)
gen_val_batches = collect_batches(gen_val_data, BATCH_SIZE, randomize=False, rotate=False)

# x, y = next(gen_val_batches)
# print(x.shape)
# print(y)
# 
# from matplotlib import pyplot as plt
# plt.imshow(x[0, 0], cmap='Greys_r')
# plt.show()

In [6]:
model = build_model(CLASS_COUNT, is_train=True)
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, None, None) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 10, None, None 100         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 20, None, None 220         conv2d_1[0][0]                   
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, 20, None, None 0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [7]:
history = model.fit_generator(generator=gen_train_batches,
                              steps_per_epoch=TRAIN_SIZE,
                              epochs=30,
                              validation_data=gen_val_batches,
                              validation_steps=VAL_SIZE,
                              callbacks=[ReduceLROnPlateau(patience=3),
                                         ModelCheckpoint(join(PATH_PROJECT, 'backups', 'weights_{epoch:02d}'))])

Epoch 1/30
5600/5600 [==============================] - 190s - loss: 0.0800 - acc: 0.9727 - val_loss: 3.3490e-04 - val_acc: 1.0000
Epoch 2/30
5600/5600 [==============================] - 156s - loss: 0.0062 - acc: 0.9987 - val_loss: 4.2460e-07 - val_acc: 1.0000
Epoch 3/30
5600/5600 [==============================] - 155s - loss: 2.9699e-04 - acc: 0.9998 - val_loss: 1.2164e-07 - val_acc: 1.0000
Epoch 4/30
5600/5600 [==============================] - 154s - loss: 0.0024 - acc: 0.9996 - val_loss: 1.3928e-04 - val_acc: 1.0000
Epoch 5/30
5600/5600 [==============================] - 153s - loss: 2.0321e-04 - acc: 1.0000 - val_loss: 7.7401e-06 - val_acc: 1.0000
Epoch 6/30
5600/5600 [==============================] - 154s - loss: 0.0013 - acc: 0.9995 - val_loss: 6.5688e-07 - val_acc: 1.0000
Epoch 7/30
5600/5600 [==============================] - 155s - loss: 5.4910e-04 - acc: 0.9998 - val_loss: 1.2070e-07 - val_acc: 1.0000
Epoch 8/30
5600/5600 [==============================] - 152s - loss: 2.

In [8]:
path_weights = join(PATH_PROJECT, 'weights')
model.save_weights(path_weights, overwrite=True)

In [9]:
def get_test_data(path_images):
    paths = map(lambda filename: join(path_images, filename), os.listdir(path_images))
    return np.array([list(map(lambda path: cv2.imread(path, cv2.IMREAD_GRAYSCALE), paths))])

In [10]:
model_predict = build_model(CLASS_COUNT, is_train=False)
model_predict.load_weights(path_weights)

In [11]:
path_test = join(PATH_DATA, 'test')
answers = {}
for filename in os.listdir(path_test):
    file_path = join(path_test, filename)
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    tensor = image.reshape((1, 1, *image.shape))
    label_categorical = model_predict.predict(tensor)
    print(label_categorical)
    answers[filename] = np.argmax(label_categorical)

[[  0.00000000e+00   1.00000000e+00   8.42038808e-25   3.91702967e-30
    2.54930231e-26   3.76771744e-30]]
[[  0.00000000e+00   1.00000000e+00   2.17316906e-25   1.11511259e-30
    1.29010500e-28   2.77527449e-27]]
[[  0.00000000e+00   4.10007754e-17   1.00000000e+00   6.03359903e-31
    6.69530206e-21   2.09350184e-30]]
[[  0.00000000e+00   9.51653138e-27   1.00000000e+00   1.27172331e-35
    6.45320042e-34   5.10834718e-35]]
[[  4.31439358e-36   1.13173792e-19   1.00000000e+00   3.59645599e-23
    3.95772474e-19   2.32675486e-23]]
[[  0.00000000e+00   1.00000000e+00   2.54185953e-22   3.49387670e-28
    1.11270251e-24   3.33936691e-20]]
[[  0.00000000e+00   1.00000000e+00   3.12399488e-26   2.01306182e-31
    3.28245629e-28   4.98518068e-27]]
[[  0.00000000e+00   1.00000000e+00   1.96523556e-25   6.48902880e-31
    1.00909099e-27   5.25185997e-27]]
[[  0.00000000e+00   1.00000000e+00   7.61040493e-26   1.09083258e-29
    3.53975740e-28   4.73147037e-25]]
[[  0.00000000e+00   1.00000

[[  0.00000000e+00   1.00000000e+00   4.94518852e-23   1.40584117e-14
    5.34630202e-26   8.88190609e-17]]
[[  0.00000000e+00   1.00000000e+00   1.01308738e-26   3.69959434e-31
    5.69753248e-28   2.07220448e-26]]
[[  0.00000000e+00   5.95889134e-17   1.00000000e+00   1.07681700e-29
    9.11373382e-21   6.12588286e-30]]
[[  1.00000000e+00   1.29829343e-31   1.09919964e-21   2.99759439e-35
    7.16292463e-14   0.00000000e+00]]
[[  0.00000000e+00   9.56805984e-30   3.09491186e-25   1.00000000e+00
    2.34152948e-13   1.93994432e-10]]
[[  7.35477731e-03   9.21126428e-08   9.92644668e-01   3.34234322e-14
    4.17412849e-07   7.73348474e-19]]
[[  0.00000000e+00   1.00000000e+00   8.52836767e-23   4.41471773e-29
    1.67014838e-23   1.17621345e-22]]
[[  0.00000000e+00   1.00000000e+00   3.97652179e-26   6.91570833e-31
    2.70868007e-27   1.07296774e-27]]
[[  0.00000000e+00   1.62835855e-21   2.00644569e-16   1.00000000e+00
    2.19202933e-12   7.40952066e-15]]
[[  0.00000000e+00   8.77313

[[  0.00000000e+00   2.18225264e-37   0.00000000e+00   1.05092938e-30
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   9.06058691e-37   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   2.82742928e-37   0.00000000e+00   4.17283000e-34
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   4.88566217e-35   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   6.47232542e-36   0.00000000e+00   1.45249155e-36
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   5.03379455e-35   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   4.63473115e-35   0.00000000e+00   8.37977378e-36
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   5.56308585e-36   0.00000000e+00   2.15384498e-36
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   1.20334668e-33   0.00000000e+00   8.47472745e-38
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   1.08614

    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   6.13810089e-35   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   1.04025384e-34   0.00000000e+00   1.83329559e-34
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   6.46261080e-34   0.00000000e+00   3.87544233e-23
    3.73751787e-35   1.00000000e+00]]
[[  0.00000000e+00   1.62713159e-33   0.00000000e+00   1.89028302e-37
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   1.81576062e-37   0.00000000e+00   4.45870347e-37
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   7.81578962e-35   0.00000000e+00   7.52448613e-36
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   7.60349649e-34   0.00000000e+00   4.27224192e-34
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   8.00387295e-30   0.00000000e+00   2.84730085e-30
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   2.31716150e-31   0.00000000e+00   6.07675374e-29
    0.00000000e+00   1.00000

    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   2.48004184e-38   0.00000000e+00   1.64576986e-35
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   1.39440434e-35   0.00000000e+00   3.41174249e-36
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   2.37648785e-37   0.00000000e+00   2.64298376e-30
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.86979642e-34
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   1.81156012e-38   0.00000000e+00   1.07959869e-34
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   1.50462452e-36   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   9.40445927e-38   0.00000000e+00   1.27993310e-36
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.48783921e-35
    0.00000000e+00   1.00000000e+00]]
[[  0.00000000e+00   5.93548271e-38   0.00000000e+00   3.08659260e-37
    0.00000000e+00   1.00000

...

[[  0.00000000e+00   1.00000000e+00   8.39929615e-21   4.63906118e-29
    4.05551650e-22   1.14514044e-25]]
[[  0.00000000e+00   1.00000000e+00   8.60919799e-24   1.03409968e-30
    3.22773648e-27   4.49496734e-26]]
[[  0.00000000e+00   6.52323719e-21   1.00000000e+00   2.35970848e-33
    2.20689001e-22   1.40501752e-30]]
[[  1.00000000e+00   5.00592958e-36   3.30078296e-24   0.00000000e+00
    1.48959049e-17   0.00000000e+00]]
[[  3.69556896e-34   1.70833709e-19   1.00000000e+00   9.11185085e-22
    6.38094536e-17   7.39406410e-18]]
[[  0.00000000e+00   4.99982622e-27   1.95739105e-21   1.00000000e+00
    1.29475879e-21   1.58418254e-21]]
[[  1.47766282e-36   1.82535517e-18   1.00000000e+00   7.15421110e-22
    3.17693570e-17   2.66648653e-24]]
[[  0.00000000e+00   4.76750461e-26   1.42570705e-22   1.00000000e+00
    7.34623703e-19   3.99257127e-19]]
[[  0.00000000e+00   5.79633233e-29   2.83420632e-17   9.99995947e-01
    2.34568660e-07   3.76084495e-06]]
[[  0.00000000e+00   7.50184

[[  1.00000000e+00   2.42240672e-38   4.83676851e-25   0.00000000e+00
    1.84396446e-18   0.00000000e+00]]
[[  0.00000000e+00   3.45037163e-33   1.00000000e+00   1.54999732e-34
    1.26858087e-32   8.81153412e-34]]
[[  0.00000000e+00   1.00000000e+00   6.76865336e-26   3.58946486e-31
    5.04615505e-28   1.31256939e-25]]
[[  0.00000000e+00   1.00000000e+00   4.37303933e-26   4.15512127e-33
    5.96158313e-29   4.19784502e-28]]
[[  0.00000000e+00   1.00000000e+00   2.71790488e-25   9.40235532e-28
    6.75517307e-23   9.33425898e-24]]
[[  0.00000000e+00   1.00000000e+00   1.88916818e-27   2.63101937e-33
    9.00503980e-30   2.78287416e-28]]
[[  0.00000000e+00   3.22179417e-20   3.44113840e-21   1.00000000e+00
    2.81222121e-13   1.96555774e-14]]
[[  0.00000000e+00   5.88073772e-27   1.28720429e-17   9.99997973e-01
    9.37975457e-08   1.88213869e-06]]
[[  0.00000000e+00   2.31817993e-37   1.00000000e+00   7.58411277e-35
    2.26293195e-31   2.02672914e-33]]
[[  0.00000000e+00   1.00000

[[  0.00000000e+00   1.00000000e+00   1.35066164e-22   2.28933944e-28
    2.42164777e-24   8.90600586e-22]]
[[  0.00000000e+00   1.00000000e+00   4.31647530e-25   3.39675220e-31
    1.30906647e-29   4.40063206e-27]]
[[  0.00000000e+00   1.00000000e+00   1.26416513e-25   5.85035489e-30
    1.27574623e-28   1.77485322e-25]]
[[  0.00000000e+00   1.27852192e-30   1.00000000e+00   7.62804306e-27
    1.47628774e-29   3.28887698e-26]]
[[  0.00000000e+00   3.13778299e-30   1.03692116e-28   5.22419092e-25
    1.00000000e+00   8.83595955e-29]]
[[  0.00000000e+00   1.00000000e+00   2.49665566e-21   3.67336285e-26
    2.44337990e-24   2.01062293e-22]]
[[  0.00000000e+00   1.00000000e+00   4.55871710e-28   9.70230997e-33
    6.15626997e-29   9.44603926e-27]]
[[  0.00000000e+00   2.11404892e-20   1.00000000e+00   6.12095096e-31
    1.78466742e-21   4.76769736e-29]]
[[  1.00000000e+00   1.15884908e-36   8.64756843e-26   0.00000000e+00
    3.33519301e-19   0.00000000e+00]]
[[  1.00000000e+00   0.00000

    6.07410982e-22   2.51996277e-22]]
[[  0.00000000e+00   1.00000000e+00   1.53583601e-27   4.14296555e-33
    1.50960287e-29   4.98591756e-29]]
[[  1.00000000e+00   0.00000000e+00   4.28751818e-29   0.00000000e+00
    2.51279355e-21   0.00000000e+00]]
[[  1.74557956e-38   6.51498085e-36   2.44679664e-29   1.00682629e-27
    1.00000000e+00   8.51018604e-34]]
[[  0.00000000e+00   1.65753019e-16   1.00000000e+00   5.68947439e-28
    3.08609279e-18   5.13948542e-27]]
[[  8.01215773e-36   1.29898933e-20   1.00000000e+00   1.18407820e-22
    1.19329864e-19   1.04102689e-21]]
[[  0.00000000e+00   1.00000000e+00   4.77135337e-22   3.34664328e-12
    3.69598255e-24   6.56116591e-15]]
[[  0.00000000e+00   1.00000000e+00   2.15286304e-25   1.57081009e-30
    5.46669256e-29   4.08581724e-27]]
[[  3.27525314e-08   1.08906512e-07   9.99999881e-01   1.57069168e-13
    1.25368671e-09   8.13024326e-18]]
[[  7.70871919e-35   3.86567763e-18   1.00000000e+00   4.91139135e-21
    1.21684541e-16   5.81392

[[  0.00000000e+00   1.00000000e+00   4.05924980e-28   4.46290301e-31
    1.86693886e-26   2.80179481e-26]]
[[  0.00000000e+00   1.00000000e+00   1.54356629e-26   7.76103798e-34
    9.76801951e-30   1.26125308e-29]]
[[  0.00000000e+00   1.58568096e-28   1.65551845e-17   9.99999166e-01
    3.87515620e-09   8.46469277e-07]]
[[  8.76998051e-35   7.04652804e-20   1.00000000e+00   2.00942676e-22
    2.66352009e-15   1.27418404e-22]]
[[  0.00000000e+00   9.36049827e-25   3.91983936e-15   1.00000000e+00
    2.02501995e-19   3.63682099e-17]]
[[  0.00000000e+00   1.00000000e+00   2.24901927e-21   2.10956336e-26
    3.12332510e-20   7.23964001e-24]]
[[  0.00000000e+00   1.00000000e+00   1.05669843e-22   5.04015054e-29
    2.19278956e-24   1.96869874e-21]]
[[  0.00000000e+00   1.49480359e-26   1.42694547e-25   1.00000000e+00
    8.01878852e-22   1.76143919e-12]]
[[  0.00000000e+00   1.00000000e+00   3.23149351e-20   1.24967669e-23
    1.79395374e-17   2.22005686e-19]]
[[  0.00000000e+00   1.00000

In [13]:
with open(join(PATH_PROJECT, 'submission.csv'), 'w') as fout:
    for filename, label in answers.items():
        print(filename, label, file=fout, sep=',')